 Author: Lourde Hajjar

 The notebook applies CNN on the focused image dataset. It builds an improved CNN model.It incorporates data augmentation (e.g., rotation, shifting, zooming, and flipping) and learning rate to enhance generalization and batch normalization layers to stabilize training and accelerate convergence.
 
 The model’s performance is evaluated using a test set and 10-fold cross-validation providing 80% accuracy

# CNN With Augmentation

## Load the Focused Dataset

In [ ]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

# Define dataset path
data_dir =r"C:\Users\lourd\OneDrive\Desktop\coursework\datasets\3_image\processed\3_foc_b"


images = []
labels = []


# Load images 
for label in os.listdir(data_dir):
    label_dir = os.path.join(data_dir, label)
    if os.path.isdir(label_dir):
        for img_file in os.listdir(label_dir):
            img_path = os.path.join(label_dir, img_file)
            img = load_img(img_path, color_mode="grayscale")
            img = load_img(img_path, target_size=(256, 256), color_mode="grayscale")
            img_array = img_to_array(img) / 255.0  # Normalize
            images.append(img_array)
            labels.append(label)


images = np.array(images)
labels = np.array(labels)

class_names = ['normal', 'malignant', 'benign']


images = images.reshape(-1, 256, 256, 1)


label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Split data into training and test sets
x_train, x_test, y_train, y_test = train_test_split(images, encoded_labels, test_size=0.2, random_state=42)


## Define the CNN Model

In [ ]:
# Define an improved CNN model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 1)),
    tf.keras.layers.BatchNormalization(), 
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),  
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),  
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')  
])


model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 254, 254, 32)      320       
                                                                 
 batch_normalization_3 (Batc  (None, 254, 254, 32)     128       
 hNormalization)                                                 
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 127, 127, 32)     0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 125, 125, 64)      18496     
                                                                 
 batch_normalization_4 (Batc  (None, 125, 125, 64)     256       
 hNormalization)                                                 
                                                      

## Apply Data Augmentation

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Apply Data augmentation 
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True
)

datagen.fit(x_train)


In [ ]:
history = model.fit(
    datagen.flow(x_train, y_train, batch_size=16),  
    validation_data=(x_test, y_test),
    epochs=30,
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)]
)


Epoch 1/30
85/85 [==============================] - 27s 301ms/step - loss: 1.6968 - accuracy: 0.5260 - val_loss: 12.7072 - val_accuracy: 0.3531
Epoch 2/30
85/85 [==============================] - 26s 305ms/step - loss: 0.7721 - accuracy: 0.6226 - val_loss: 22.5731 - val_accuracy: 0.3531
Epoch 3/30
85/85 [==============================] - 28s 330ms/step - loss: 0.6713 - accuracy: 0.6724 - val_loss: 24.6629 - val_accuracy: 0.3531
Epoch 4/30
85/85 [==============================] - 28s 324ms/step - loss: 0.5967 - accuracy: 0.7132 - val_loss: 17.9497 - val_accuracy: 0.3531
Epoch 5/30
85/85 [==============================] - 27s 318ms/step - loss: 0.6149 - accuracy: 0.6828 - val_loss: 7.5537 - val_accuracy: 0.4095
Epoch 6/30
85/85 [==============================] - 26s 310ms/step - loss: 0.5489 - accuracy: 0.7244 - val_loss: 5.3693 - val_accuracy: 0.4273
Epoch 7/30
85/85 [==============================] - 27s 312ms/step - loss: 0.5587 - accuracy: 0.7363 - val_loss: 2.9140 - val_accuracy: 0.

## Evaluate the Model

In [21]:
test_loss, test_acc = model.evaluate(x_test,  y_test, verbose=0)
print('\nTest accuracy:', test_acc)


Test accuracy: 0.8041542768478394


## Print Classification Report

In [ ]:
from sklearn.metrics import f1_score, classification_report
import numpy as np

# Make Predictions on the Test Set
y_pred_prob = model.predict(x_test)

# Convert probabilities to class predictions
y_pred = np.argmax(y_pred_prob, axis=1)


if len(y_test.shape) > 1 and y_test.shape[1] > 1:
    
    y_true = np.argmax(y_test, axis=1)
else:
    
    y_true = y_test

# Calculate F1 Score
f1_macro = f1_score(y_true, y_pred, average='macro')
f1_weighted = f1_score(y_true, y_pred, average='weighted')

# Print Results
print("Test Accuracy:", np.mean(y_true == y_pred))
print("F1 Score (Macro):", f1_macro)
print("F1 Score (Weighted):", f1_weighted)

# Print Classification Report
print("\nClassification Report:")
print(classification_report(y_true, y_pred))


11/11 [==============================] - 6s 190ms/step
Test Accuracy: 0.8041543026706232
F1 Score (Macro): 0.8002914700162407
F1 Score (Weighted): 0.8050834225391164

Classification Report:
              precision    recall  f1-score   support

           0       0.76      0.64      0.70       118
           1       1.00      0.97      0.98       119
           2       0.66      0.80      0.72       100

    accuracy                           0.80       337
   macro avg       0.81      0.80      0.80       337
weighted avg       0.81      0.80      0.81       337



## Print 10-Fold Cross-Validation Results:

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
import numpy as np

# Initialize KFold cross-validation with 10 splits
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Lists to store metrics for each fold
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []
roc_auc_scores = []
true_positive_rates = []
false_positive_rates = []

# Loop over each fold
for train_index, val_index in kf.split(x_train):
   
    x_val_fold = x_train[val_index]
    y_val_fold = y_train[val_index]

    # Predict on the validation set
    y_val_pred_prob = model.predict(x_val_fold)
    y_val_pred = np.argmax(y_val_pred_prob, axis=1)

    # Calculate and store metrics
    accuracy = accuracy_score(y_val_fold, y_val_pred)
    precision = precision_score(y_val_fold, y_val_pred, average='macro')
    recall = recall_score(y_val_fold, y_val_pred, average='macro')
    f1 = f1_score(y_val_fold, y_val_pred, average='macro')

   
    if len(np.unique(y_val_fold)) == 2:
        roc_auc = roc_auc_score(y_val_fold, y_val_pred_prob[:, 1])
        roc_auc_scores.append(roc_auc)

    # Calculate TP and FP rates from confusion matrix
    cm = confusion_matrix(y_val_fold, y_val_pred)
    tp_rate = cm[1, 1] / (cm[1, 1] + cm[1, 0]) if (cm[1, 1] + cm[1, 0]) > 0 else 0
    fp_rate = cm[0, 1] / (cm[0, 1] + cm[0, 0]) if (cm[0, 1] + cm[0, 0]) > 0 else 0

    true_positive_rates.append(tp_rate)
    false_positive_rates.append(fp_rate)
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)

# Calculate the average of each metric across all folds
print("10-Fold Cross-Validation Results:")
print("Average Accuracy:", np.mean(accuracy_scores))
print("Average Precision:", np.mean(precision_scores))
print("Average Recall:", np.mean(recall_scores))
print("Average F1 Score:", np.mean(f1_scores))
print("Average TP Rate:", np.mean(true_positive_rates))
print("Average FP Rate:", np.mean(false_positive_rates))

if roc_auc_scores:
    print("Average ROC AUC Score:", np.mean(roc_auc_scores))


5/5 [==============================] - 1s 221ms/step
10-Fold Cross-Validation Results:
Average Accuracy: 0.7905030403537866
Average Precision: 0.7995164873463204
Average Recall: 0.7907980083113697
Average F1 Score: 0.7892427440903667
Average TP Rate: 0.9642160982648786
Average FP Rate: 0.0
